In [2]:
%matplotlib ipympl
# %matplotlib inline
import matplotlib.pyplot as plt
import tensorflow as tf
from utils import create_session_config
import tensorflow_probability as tp
import numpy as np
from models.utils import covariance,correlation_matrix,tf_cov,assert_cov_positive_definite,corr,print_tensor,print_tensors,constrain_cdf
from models.nn_pdf_common import transform_x,density_estimator,create_pdf_layer_mv,create_partially_monotone_dense_layer,create_monotone_dense_layer,create_positive_weights
from matplotlib import pyplot as plt
from scipy.stats import multivariate_normal
from flags import FLAGS
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import os
np.set_printoptions(threshold=np.inf)

In [87]:
# def cdf_transform(x, y, params, positive_transform):
#     x_transform = transform_x(params, x)
  
#     ys = []
#     xys = []
#     for i in range(y.shape[1].value):
#         ys.append(tf.slice(y, [0, i], [-1, 1]) )
#         xys.append(tf.concat([x_transform, ys[-1]], axis=1))
    
#     activation1 = tf.nn.sigmoid
#     activation2 = tf.nn.relu

#     arch2 = params['arch2']
#     arch3 = params['arch3']

#     with tf.variable_scope("cdf_transform"):
#         for i in range(len(ys)):
#             with tf.variable_scope("y_comp_%d"%i):
#                 with tf.variable_scope("xy%d_partially_monotone"%i):
#                     xys[i] = create_partially_monotone_dense_layer(xys[i], arch2[0], x_transform.shape[-1].value,
#                                                                 1, activation=activation1, positive_transform=positive_transform)

#                 for j, units in enumerate(arch2[1:-1]):
#                     with tf.variable_scope("xy%d_l_%d"%(i,j)):
#                         xys[i] = create_monotone_dense_layer(xys[i], units, activation=activation1, positive_transform=positive_transform)

#                 with tf.variable_scope("xy1_l_%d"%len(arch2)):
#                     xy1_prod = create_monotone_dense_layer(xy1, arch2[-1], activation=activation1, positive_transform=positive_transform)

#                 with tf.variable_scope("xy2_partially_monotone"):
#                     xy2 = create_partially_monotone_dense_layer(xy2, arch2[0], x_transform.shape[-1].value,
#                                                                 1, activation=activation1, positive_transform=positive_transform)

#                 for i, units in enumerate(arch2[1:-1]):
#                     with tf.variable_scope("xy2_l_%d"%i):
#                         xy2 = create_monotone_dense_layer(xy2, units, activation=activation1, positive_transform=positive_transform)

#                 with tf.variable_scope("xy2_l_%d"%len(arch2)):
#                     xy2_prod = create_monotone_dense_layer(xy2, arch2[-1], activation=activation1, positive_transform=positive_transform)
  

#         with tf.variable_scope("xy_prod"):
#             xy_prefinal = tf.multiply(xy1_prod, xy2_prod)
#             xy_prefinal = create_monotone_dense_layer(xy_prefinal, arch3[0], activation=activation2,
#                                                       positive_transform=positive_transform)

#         for i, units in enumerate(arch3[1:]):
#             with tf.variable_scope("xy_prefinal_l_%d"%i):
#                 xy_prefinal = create_monotone_dense_layer(xy_prefinal, units, activation=activation2,
#                                                       positive_transform=positive_transform)

#         return xy_prefinal,create_monotone_dense_layer(xy_prefinal, 1, activation=activation2,
#                                            positive_transform=positive_transform)

def cdf_transform(x, y, params, positive_transform):
    x_transform = transform_x(params, x)

    y1 = tf.slice(y, [0, 0], [-1, 1])

    y2 = tf.slice(y, [0, 1], [-1, 1])

    xy1 = tf.concat([x_transform, y1], axis=1)
    xy2 = tf.concat([x_transform, y2], axis=1)

    activation1 = tf.nn.sigmoid
    activation2 = tf.nn.relu

    arch2 = params['arch2']
    arch3 = params['arch3']
    #     relu_bias_initializer = float(params["relu_bias_initializer"])

    with tf.variable_scope("cdf_transform"):

        with tf.variable_scope("xy1_partially_monotone"):
            xy1 = create_partially_monotone_dense_layer(xy1, arch2[0], x_transform.shape[-1].value,
                                                        1, activation=activation1,
                                                        positive_transform=positive_transform)

        for i, units in enumerate(arch2[1:-1]):
            with tf.variable_scope("xy1_l_%d" % i):
                xy1 = create_monotone_dense_layer(xy1, units, activation=activation1,
                                                  positive_transform=positive_transform)

        with tf.variable_scope("xy1_l_%d" % len(arch2)):
            xy1_prod = create_monotone_dense_layer(xy1, arch2[-1], activation=activation1,
                                                   positive_transform=positive_transform)

        #         with tf.variable_scope("xy1_add_l_%d"%len(arch2)):
        #             xy1_add = create_monotone_dense_layer(xy1, arch2[-1], activation=activation1, positive_transform=positive_transform)

        with tf.variable_scope("xy2_partially_monotone"):
            xy2 = create_partially_monotone_dense_layer(xy2, arch2[0], x_transform.shape[-1].value,
                                                        1, activation=activation1,
                                                        positive_transform=positive_transform)

        for i, units in enumerate(arch2[1:-1]):
            with tf.variable_scope("xy2_l_%d" % i):
                xy2 = create_monotone_dense_layer(xy2, units, activation=activation1,
                                                  positive_transform=positive_transform)

        with tf.variable_scope("xy2_l_%d" % len(arch2)):
            xy2_prod = create_monotone_dense_layer(xy2, arch2[-1], activation=activation1,
                                                   positive_transform=positive_transform)

        #         with tf.variable_scope("xy2_add_l_%d"%len(arch2)):
        #             xy2_add = create_monotone_dense_layer(xy2, arch2[-1], activation=activation1, positive_transform=positive_transform)

        with tf.variable_scope("xy_prod"):
            xy_prefinal = tf.multiply(xy1_prod, xy2_prod)
            #             tf.concat([xy_prefinal, xy1_add,xy2_add], axis=1)
            xy_prefinal = create_monotone_dense_layer(xy_prefinal, arch3[0], activation=activation2,
                                                      positive_transform=positive_transform)

        for i, units in enumerate(arch3[1:]):
            with tf.variable_scope("xy_prefinal_l_%d" % i):
                xy_prefinal = create_monotone_dense_layer(xy_prefinal, units, activation=activation2,
                                                          positive_transform=positive_transform)

        return create_monotone_dense_layer(xy_prefinal, 1, activation=activation2,
                                                        positive_transform=positive_transform)
    
def create_pdf_layer(cdf, y):
    with tf.name_scope("pdf_layer"):
        assert cdf.shape[0].value == y.shape[0].value
        assert y.shape[-1].value == 1
        assert cdf.shape[-1].value == 1
        # because it is list and cdf is only 1 element
        return tf.gradients(cdf, y)[0]
    
def create_pdf_layer_mv(cdf, y_components):
    gradients = cdf
    for i, y in enumerate(y_components):
        with tf.name_scope("pdf_layer_mv_%d"%i):
            gradients = create_pdf_layer(gradients, y)
    return gradients

def cdf_transform_normalizd(x, y, params, positive_transform, mode):
    with tf.variable_scope("extreme_vals",reuse=tf.AUTO_REUSE):
        y_max_values_vars = tf.get_variable("y_max_values_var", trainable=False, dtype=tf.float32,
                                            shape=(1, y.shape[-1].value), initializer=tf.constant_initializer(np.nan))
        y_min_values_vars = tf.get_variable("y_min_values_var", trainable=False, dtype=tf.float32,
                                            shape=(1, y.shape[-1].value), initializer=tf.constant_initializer(np.nan))

        if mode == tf.estimator.ModeKeys.TRAIN:
            y_max_values = tf.reduce_max(y, axis=0, keepdims=True)
            y_max_values_vars = tf.cond(tf.reduce_all(tf.is_nan(y_max_values_vars)), \
                  lambda: tf.assign(y_max_values_vars, y_max_values), \
                  lambda: tf.assign(y_max_values_vars, tf.reduce_max(tf.concat([y_max_values, y_max_values_vars], axis=0),axis=0, keepdims=True)))

            y_min_values = tf.reduce_min(y, axis=0, keepdims=True)
            y_min_values_vars = tf.cond(tf.reduce_all(tf.is_nan(y_min_values_vars)), \
                  lambda: tf.assign(y_min_values_vars, y_min_values), \
                  lambda: tf.assign(y_min_values_vars, tf.reduce_max(tf.concat([y_min_values, y_min_values_vars], axis=0),axis=0, keepdims=True)))

    with tf.variable_scope("cdf", reuse=tf.AUTO_REUSE):
        xy_prefinal,non_normalized_cdf = cdf_transform(x, y, params, positive_transform)
        _,cdf_normalization = cdf_transform(x, tf.tile(y_max_values_vars, [tf.shape(x)[0], 1]),
                                                       params, positive_transform)
    return xy_prefinal,non_normalized_cdf,cdf_normalization,y_min_values_vars, y_max_values_vars,print_tensor(tf.div(non_normalized_cdf, tf.stop_gradient(cdf_normalization)), name="cdf")

In [86]:
tf.reset_default_graph()
tf.set_random_seed(10)
global_step_tensor = tf.Variable(0, trainable=False, name='global_step')
corr_coef = 0.7
corr_ = np.array([[1.0,corr_coef],[corr_coef,1.0]], dtype=np.float32)
std_dev = np.array([[7.0,0.0],[0,9.0]], dtype=np.float32)
true_cov = np.matmul(std_dev, np.matmul(corr_,std_dev))
print("true_cov: %s"%true_cov)

y_size = 2
samples = 1000

if y_size == 2:
    data_gen = tp.distributions.MultivariateNormalFullCovariance(loc=tf.constant([0.0] * 2),
                                                                 covariance_matrix=tf.constant(true_cov))
    sample_op = data_gen.sample([samples])
elif y_size==1:
    data_gen = tp.distributions.Normal(loc=tf.constant(0.0), scale=3.0)
    sample_op = tf.reshape(data_gen.sample([samples]), [-1,1])
else:
    raise ValueError("wrong size")
    

dataset_gen = tf.data.Dataset.from_tensors(sample_op)
y_ph=tf.placeholder(dtype=tf.float32, shape=[None,2])
dataset_ph = tf.data.Dataset.from_tensors(y_ph)

iter = tf.data.Iterator.from_structure(dataset_gen.output_types,output_shapes=(tf.TensorShape([None,2])))

y=iter.get_next()
y=tf.Print(y,[tf.train.get_global_step(),y], message="==========================================================start: ", summarize=1)
x_size, x= 1 , tf.ones(shape=(tf.shape(y)[0],1), dtype=tf.float32)

gen_init_op = iter.make_initializer(dataset_gen)
ph_init_op = iter.make_initializer(dataset_ph)

mode=tf.estimator.ModeKeys.TRAIN
params={}
params['positive_transform'] = "square"
params['learning_rate'] = 0.0001
params['arch1'] = [1]
params['arch2'] = [10]
params['arch3'] = [10]
# params['relu_bias_initializer']=100.0

positive_transform = params['positive_transform']
learning_rate = params["learning_rate"]

if y is None or y.shape[-1].value != 2:
    raise NotImplementedError

y_components = [tf.slice(y, [0,i],[-1,1]) for i in range(y_size)]
y_components_combined = tf.concat(y_components, axis=1)


xy_prefinal,cdf_non_normalized, cdf_normalization, y_min,y_max,cdf = cdf_transform_normalizd(x, y_components_combined, params, positive_transform, mode)

grads = create_pdf_layer_mv(cdf, y_components)

pdf = tf.maximum(grads,1e-24) 
log_likelihood = tf.log(pdf, name="log_likelihood")

loss = tf.negative(tf.reduce_mean(log_likelihood), name="loss")

dlosss_dpdf = tf.gradients(loss,grads)[0]

loss = tf.Print(loss,[tf.train.get_global_step(),loss], message="loss: ", summarize=1000)
true_loss = tf.negative(tf.reduce_mean(data_gen.log_prob(y)), name="loss_true")

optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
# optimizer = tf.train.AdagradOptimizer(learning_rate=0.1)

train_op = optimizer.minimize(loss, global_step=tf.train.get_global_step())

# with tf.name_scope("train_op"):
#     trainables = tf.trainable_variables()

#     grads = tf.gradients(loss, trainables)

# #     grads, _ = tf.clip_by_global_norm(grads, clip_norm=1)
# #     grads, _ = tf.clip_by_value(grads, clip_value_max=1, clip_value_min=-1)

#     grads_printed = []
#     grads_clipped = []
#     for grad, var in zip(grads, trainables):
# #         print(var.name, var, grad.name, grad)
# #         grads_printed.append(tf.Print(grad,[tf.train.get_global_step() ,grad], message="grad_%s: "%var.name, summarize=100))
#         grads_clipped.append(tf.clip_by_value(grad, -100, 100))
# #         grads_clipped.append(tf.clip_by_norm(grad, 0.01))
    
# #     optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  
#     train_op = optimizer.apply_gradients(zip(grads_clipped, trainables), global_step=tf.train.get_global_step())

delta=30
y_centre_1 = 0
y_centre_2 = 0
points=100
y_1 = np.linspace(y_centre_1-delta,y_centre_1+delta,points)
y_2 = np.linspace(y_centre_2-delta,y_centre_2+delta,points)

y_grid = np.meshgrid(y_1,y_2)

y_grid_flat = np.concatenate(list(map(lambda a: a.reshape(-1,1), y_grid)), axis=1)

def show_cdf():
    session.run(ph_init_op, feed_dict={y_ph:y_grid_flat})
    cdf_vals,pdf_vals = session.run([cdf,pdf])
    true_cdf_vals=multivariate_normal.cdf(y_grid_flat, cov=true_cov)
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_wireframe(y_grid[0], y_grid[1],cdf_vals.reshape(points,points),cmap=cm.coolwarm)
    ax.plot_wireframe(y_grid[0], y_grid[1],true_cdf_vals.reshape(points,points),cmap=cm.gray)
    plt.title("cdf")
    plt.xlabel("y1")
    plt.ylabel("y2")
    plt.show() 
    
#     fig = plt.figure(figsize=(10,6))
#     ax = fig.gca(projection='3d')
#     ax.set_xlabel("y1");
#     ax.set_ylabel("y2");
#     ax.set_zlabel("cdf");
    
#     surf=ax.plot_surface(y_grid[0],y_grid[1],cdf_vals.reshape(points,points),cmap=cm.coolwarm, linewidth=0, antialiased=False)
    
    
#     ax.plot_surface(y_grid[0],y_grid[1],true_cdf_vals.reshape(points,points),cmap=cm.gray, linewidth=0, antialiased=False)
    
#     ax.zaxis.set_major_locator(LinearLocator(10))
#     ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

#     # Add a color bar which maps values to colors.
#     fig.colorbar(surf, shrink=0.5, aspect=5)
    
#     plt.show()
    
    true_pdf_vals=multivariate_normal.pdf(y_grid_flat, cov=true_cov)
    
    print("model pdf integral: %f"%(((2*delta)**2)*np.sum(pdf_vals)/len(pdf_vals)))
    print("true pdf integral: %f"%(((2*delta)**2)*np.sum(true_pdf_vals)/len(true_pdf_vals)))
#     print("pdf_vals: %r"%pdf_vals[:10])
#     print("true_pdf_vals: %r"%true_pdf_vals[:10])

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_wireframe(y_grid[0], y_grid[1],pdf_vals.reshape(points,points))
    plt.title("model pdf")
    plt.xlabel("y1")
    plt.ylabel("y2")
    plt.show()
    
    fig = plt.figure(figsize=(10,6))
    extent = [y_1[0], y_1[-1], y_2[0], y_2[-1]]
    plt.ylabel('y2')
    plt.xlabel('y1')
    im=plt.imshow(pdf_vals.reshape(points,points), extent=extent)
    plt.title("model pdf")
    plt.colorbar(im);
    plt.show()
    
    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    ax.plot_wireframe(y_grid[0], y_grid[1],true_pdf_vals.reshape(points,points))
    plt.title("true pdf")
    plt.xlabel("y1")
    plt.ylabel("y2")
    plt.show() 
    
    fig = plt.figure(figsize=(10,6))
    extent = [y_1[0], y_1[-1], y_2[0], y_2[-1]]
    plt.ylabel('y2')
    plt.xlabel('y1')
    plt.title("true pdf")
    im=plt.imshow(true_pdf_vals.reshape(points,points), extent=extent)
    plt.colorbar(im);
    plt.show()    
    

def train_loop(iters, tain_op):
    session.run(gen_init_op)
    print("step, loss and true_loss: ",session.run([tf.train.get_global_step(),loss,true_loss]))
  
    for step in range(iters):  
        session.run(gen_init_op)
        step_val,loss_val,_=session.run([tf.train.get_global_step(),loss,tain_op])
        if step % 100 ==0:
            print("step: {step}, loss: {loss}".format(step=step_val,loss=loss_val))


#                 variables_names = [v.name for v in tf.trainable_variables()]
#                 values = session.run(variables_names)
#                 for k, v in zip(variables_names, values):
#                     file.write("Variable: {var} , shape: {shape}, val: {val}\n".format(var=k,shape=v.shape,val=v))

#         file.write("min cdf: %.10f\n"%min_cdf_val)
#         file.write("max cdf: %.10f\n"%max_cdf_val)
#         file.write("min pdf: %.10f\n"%min_pdf_val)
#         file.write("max pdf: %.10f\n"%max_pdf_val)
#             print("cdf_non_normalized_vals ",cdf_non_normalized_vals)
#             print("cdf_normalization_vals ",cdf_normalization_vals)
#             print("xy_prefinal: ", xy_prefinal_vals)
#         file.write("y min: %r\n"%y_min_vals)
#         file.write("y max: %r\n"% y_max_vals)

#         session.run(ph_init_op, feed_dict={y_ph:y_min_vals})
#         file.write("CDF at min: %f\n"%session.run(cdf)) 

#         session.run(ph_init_op, feed_dict={y_ph:y_max_vals})
#         file.write("CDF at max: %f\n"%session.run(cdf)) 

#                 file.write("grads_vals: %r\n"%grads_vals) 

#         file.flush()
#         os.fsync(file)

    #             y_range= np.c_[np.linspace(y_min_vals[0][0],y_max_vals[0][0],10).reshape(-1,1),np.linspace(y_min_vals[0][1],y_max_vals[0][1],10).reshape(-1,1)]
    #             session.run(ph_init_op, feed_dict={y_ph:y_range})
    #             print("y range: ",y_range) 
    #             print("CDF range: ",session.run(cdf)) 

    #             y1_test = np.sort(np.random.uniform(-10,10,100))
    #             y2_test = np.sort(np.random.uniform(-10,10,100))

    #             session.run(ph_init_op, feed_dict={y_ph:np.c_[y1_test,y2_test]})
    #             print("cdf_increasing: ", np.all(np.diff(session.run(cdf)) >= 0) ) 

    #             y2_test = np.tile(np.random.uniform(-10,10,1), 100) 
    #             y1_test = np.sort(np.random.uniform(-10,10,100))
    #             session.run(ph_init_op, feed_dict={y_ph:np.c_[y1_test,y2_test]})
    #             print("cdf1_increasing: ", np.all(np.diff(session.run(cdf)) >= 0) ) 

    #             y1_test = np.tile(np.random.uniform(-10,10,1), 100) 
    #             y2_test = np.sort(np.random.uniform(-10,10,100))
    #             session.run(ph_init_op, feed_dict={y_ph:np.c_[y1_test,y2_test]})
    #             print("cdf2_increasing: ", np.all(np.diff(session.run(cdf)) >= 0) ) 
        
            
    session.run(gen_init_op)
    print("true_loss: ",session.run(true_loss))
    session.run(gen_init_op)
    print("loss: ",session.run(loss))
    show_cdf()

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))) as session:
    session.run(tf.global_variables_initializer())
    train_loop(30000, train_op)


true_cov: [[49.       44.1     ]
 [44.100002 81.      ]]
step, loss and true_loss:  [0, 9.849984, 6.667027]
step: 1, loss: 9.781632423400879
step: 101, loss: 9.136486053466797
step: 201, loss: 8.553621292114258
step: 301, loss: 8.248153686523438
step: 401, loss: 8.042720794677734
step: 501, loss: 7.977960109710693
step: 601, loss: 7.873007297515869
step: 701, loss: 7.74100399017334
step: 801, loss: 7.676098346710205
step: 901, loss: 7.535309314727783
step: 1001, loss: 7.524650573730469
step: 1101, loss: 7.48321008682251
step: 1201, loss: 7.351883888244629
step: 1301, loss: 7.278109073638916
step: 1401, loss: 7.245182991027832
step: 1501, loss: 7.20064115524292
step: 1601, loss: 7.14904260635376
step: 1701, loss: 7.099614143371582
step: 1801, loss: 7.052713871002197
step: 1901, loss: 7.116256237030029
step: 2001, loss: 7.01458740234375
step: 2101, loss: 6.999449729919434
step: 2201, loss: 6.9806952476501465
step: 2301, loss: 6.973982334136963
step: 2401, loss: 6.988390922546387
step: 25

/home/pawel/tensorflow/lib/python3.5/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/pawel/tensorflow/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

model pdf integral: 0.976090
true pdf integral: 0.979348


/home/pawel/tensorflow/lib/python3.5/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/pawel/tensorflow/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


...

FigureCanvasNbAgg()

/home/pawel/tensorflow/lib/python3.5/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/pawel/tensorflow/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

/home/pawel/tensorflow/lib/python3.5/site-packages/matplotlib/pyplot.py:513: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)
/home/pawel/tensorflow/lib/python3.5/site-packages/ipywidgets/widgets/widget.py:411: DeprecationWarning: Passing unrecoginized arguments to super(FigureCanvasNbAgg).__init__().
__init__() missing 1 required positional argument: 'figure'
This is deprecated in traitlets 4.2.This error will be raised in a future release of traitlets.
  super(Widget, self).__init__(**kwargs)


FigureCanvasNbAgg()

In [70]:
x=tf.constant(1e-24)
l=tf.log(x)
g= tf.gradients(l,x)[0]
# z= tf.gradients(g,x)[0]

with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))) as session:
    session.run(tf.global_variables_initializer())
    session.run(l)
    session.run(g)
#     session.run(z)

-55.262043

1e+24

In [22]:
tf.reset_default_graph()
a=tf.get_variable(initializer=1.0,name="a")
b=tf.assign(a,2.0)
with tf.Session(config=tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.5))) as session:
    session.run(tf.global_variables_initializer())
    session.run([a,b])

[2.0, 2.0]